In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
dish = pd.read_csv("../01_openrefine/Dish_clean.csv.gz", compression='gzip')
dish.fillna('', inplace=True)

In [3]:
def freq(col, printThis = True, printCount = -1):
    result = {}
    for row in col:
        if row in result: result[row] += 1
        else: result[row] = 1
    
    if printThis:
        sort = []
        for (k, v) in result.items(): sort.append((k,v))
        sort = sorted(sort, key = lambda x: -x[1])
        for el in sort:
            print(str(el[0]) + "\t" + str(el[1]))
            printCount = printCount - 1
            if printCount == 0: break
                
    return result

In [4]:
dish.head()

,id,name,name_cluster,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,1,Consomme printaniere royal,Consomme printaniere royal,8,8,1897,1927,0.2,0.4
1,2,Chicken gumbo,Chicken gumbo,111,117,1895,1960,0.1,0.8
2,3,Tomato aux croutons,Tomato aux croutons,13,13,1893,1917,0.25,0.4
3,4,Onion au gratin,Onion au gratin,41,41,1900,1971,0.25,1
4,5,St. Emilion,St. Emilion,66,68,1881,1981,0,18


In [5]:
words = { "a la": re.compile(r'\bA La\b'),
          "of": re.compile(r'\bOf\b'),
         "in": re.compile(r'\bIn\b'),
         "every": re.compile(r'\bEvery\b'),
         "with": re.compile(r'\bWith\b'),
        "or": re.compile(r'\bOr\b'),
         "on": re.compile(r'\bOn\b'),
         "per": re.compile(r'\bPer\b'),
         "and": re.compile(r'\bAnd\b')
        }

def cleanDish(x):
    # Convert to title case
    x = x.title()
    
    # Title is a little too greedy so correct
    for (k,v) in words.items():
        # Apply the pattern stored in the values of the dictionary,
        # replacing it by the corresponding key.
        # e.g. "A La" gets replaced by "a la" because "a la" => r'\bA La\b'
        x = v.sub(k, x)
   
    return x

In [6]:
dish['name'] = dish['name'].apply(cleanDish)

In [7]:
r = freq(dish['name'], printCount = 10)

Cold Roast Beef	14
Potatoes Hashed in Cream	13
Roast Beef Sandwich	12
Sirloin Steak with Mushrooms	11
Hot Roast Beef Sandwich	11
Corned Beef Sandwich	11
Clams: Little Necks	11
Club Sandwich	10
Lobster Salad	10
American Cheese	10


In [8]:
dish[dish['name'] == 'Cold Roast Beef']

,id,name,name_cluster,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
5344,6512,Cold Roast Beef,Cold roast beef,669,671,1851,1985,0,75
299001,376607,Cold Roast Beef,Cold Roast Beef,77,78,1864,1959,0.05,5
304301,382434,Cold Roast Beef,Cold Roast Beef,1,1,1916,1916,,
305568,383832,Cold Roast Beef,Cold Roast beef,21,22,1866,1958,0.4,2.65
305612,383907,Cold Roast Beef,Cold Roast beef,1,1,1920,1920,1,1
330960,411988,Cold Roast Beef,cold roast beef,3,3,1913,1917,0.1,0.8
339698,422360,Cold Roast Beef,COLD ROAST BEEF,2,2,1915,1915,,
353955,438553,Cold Roast Beef,Cold Roast beef,2,2,1901,1914,0.3,0.55
360054,445355,Cold Roast Beef,Cold roast beef,2,2,1914,1914,0.5,0.5
373809,460027,Cold Roast Beef,cold Roast Beef,1,1,1916,1916,0.55,0.55


In [9]:
def dedupDish():
    new_dish = dish.copy()
    
    freqs = freq(new_dish['name'], printThis = False)

    nextID = max(new_dish['id']) + 1

    menus_appeared = {}
    times_appeared = {}
    first_appeared = {}
    last_appeared = {}
    lowest_price = {}
    highest_price = {}
    replacedKey = {}
    freqs2 = {}
    for (k, v) in freqs.items():
        if v > 1:
            freqs2[k] = v
            menus_appeared[k] = 0
            times_appeared[k] = 0
            first_appeared[k] = None
            last_appeared[k] = None
            lowest_price[k] = None
            highest_price[k] = None
            replacedKey[k] = nextID
            nextID = nextID + 1
    freqs = freqs2
    del(freqs2)
    
    # Sometimes these fields are blank. Change to NaN so that aggregates work better
    new_dish['first_appeared'].replace(0, np.nan, inplace=True)
    new_dish['last_appeared'].replace(0, np.nan, inplace=True)
    new_dish['lowest_price'].replace(r'^\s*$', np.nan, regex=True, inplace=True)
    new_dish['highest_price'].replace(r'^\s*$', np.nan, regex=True, inplace=True)
    
    # These are used for logistics--which foreign keys do we need to update
    # and which rows do we need to delete
    new_rows = []
    delete_me = []
    replaceKeys = {}
    
    # Helper function with aggregate
    def rolling_min(m, x):
        if m is None: return x
        elif x < m: return x
        else: return m
    
    def rolling_max(m, x):
        if m is None: return x
        elif x > m: return x
        else: return m
    
    counter = 0
    for (index, row) in new_dish.iterrows():
        counter = counter + 1
        if counter % 10000 == 0: print(counter)
        if row['name'] in freqs:
            N = row['name']
            menus_appeared[N] += row['menus_appeared']
            times_appeared[N] += row['times_appeared']
            first_appeared[N] = rolling_min(first_appeared[N], row['first_appeared'])
            last_appeared[N] = rolling_max(last_appeared[N], row['last_appeared'])
            lowest_price[N] = rolling_min(lowest_price[N], row['lowest_price'])
            highest_price[N] = rolling_max(highest_price[N], row['highest_price'])
            delete_me.append(index)
            replaceKeys[row['id']] = replacedKey[N]

    new_dish.drop(delete_me, inplace=True)

    new_rows = pd.DataFrame({
        "id": [replacedKey[k] for k in freqs.keys()],
        "name": [k for k in freqs.keys()],
        "name_cluster": [k for k in freqs.keys()],
        "menus_appeared": [menus_appeared[k] for k in freqs.keys()],
        "times_appeared": [times_appeared[k] for k in freqs.keys()],
        "first_appeared": [first_appeared[k] for k in freqs.keys()],
        "last_appeared": [last_appeared[k] for k in freqs.keys()],
        "lowest_price": [lowest_price[k] for k in freqs.keys()],
        "highest_price": [highest_price[k] for k in freqs.keys()]})
    
    # Hack to force new_rows to have same types as original data frame
    new_rows = new_rows.astype(new_dish.dtypes.to_dict())
    new_dish = new_dish.append(new_rows, ignore_index = True)        
    new_dish['first_appeared'] = new_dish['first_appeared'].astype('Int64')
    new_dish['last_appeared'] = new_dish['last_appeared'].astype('Int64')

    return (new_dish, replaceKeys)

(dish, replaceKeys) = dedupDish()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000


In [10]:
dish[dish['name'] == 'Cold Roast Beef']

,id,name,name_cluster,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
368125,517858,Cold Roast Beef,Cold Roast Beef,798,802,1851,1985,0.0,75.0


In [11]:
r = freq(dish['name'], printCount = 10)

St. Emilion	1
Cream of New Asparagus, Croutons	1
Fresh Lobsters in every Style	1
India Chutney	1
Pate De Foies-Gras	1
Pomard	1
Brook Trout, Mountain Style	1
Whitebait, Sauce Tartare	1
Claremont Planked Shad	1
Cerealine with Milk	1


In [12]:
dish.to_csv("p02_dish_cleaned.csv.gz", compression='gzip', index = False)

In [13]:
menuItem = pd.read_csv("../01_openrefine/MenuItem_clean.csv.gz", compression='gzip')
menuItem['dish_id'] = menuItem['dish_id'].astype('Int64')

In [14]:
menuItem.head()

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,1,1389,0.4,NaN,1,2011-03-28 15:00:44 UTC,2011-04-19 04:33:15 UTC,0.111429,0.254735
1,2,1389,0.6,NaN,2,2011-03-28 15:01:13 UTC,2011-04-19 15:00:54 UTC,0.438571,0.254735
2,3,1389,0.4,NaN,3,2011-03-28 15:01:40 UTC,2011-04-19 19:10:05 UTC,0.140000,0.261922
3,4,1389,0.5,NaN,4,2011-03-28 15:01:51 UTC,2011-04-19 19:07:01 UTC,0.377143,0.262720
4,5,3079,0.5,1.0,5,2011-03-28 15:21:26 UTC,2011-04-13 15:25:27 UTC,0.105714,0.313178


In [15]:
def replaceDishID(x):
    if x in replaceKeys: return replaceKeys[x]
    else: return x

temp = menuItem['dish_id'].apply(replaceDishID).astype('Int64')
print("{0} rows changed, {1} not changed.".format(sum(temp != menuItem['dish_id']), sum(temp == menuItem['dish_id'])))
menuItem['dish_id'] = temp

738797 rows changed, 593929 not changed.


In [16]:
menuItem.head()

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,1,1389,0.4,NaN,515678,2011-03-28 15:00:44 UTC,2011-04-19 04:33:15 UTC,0.111429,0.254735
1,2,1389,0.6,NaN,515679,2011-03-28 15:01:13 UTC,2011-04-19 15:00:54 UTC,0.438571,0.254735
2,3,1389,0.4,NaN,515680,2011-03-28 15:01:40 UTC,2011-04-19 19:10:05 UTC,0.140000,0.261922
3,4,1389,0.5,NaN,515681,2011-03-28 15:01:51 UTC,2011-04-19 19:07:01 UTC,0.377143,0.262720
4,5,3079,0.5,1.0,5,2011-03-28 15:21:26 UTC,2011-04-13 15:25:27 UTC,0.105714,0.313178


In [17]:
menuItem.to_csv("p02_MenuItem_cleaned.csv.gz", compression='gzip', index = False)